In [1]:
import torch
from torch import nn
from pathlib import Path
import numpy as np
import seaborn as sns
import scipy.io as spio
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from modules import optimize_ML, optimize_ELBO

import os

In [ ]:
# load sample trajectory
sim_idx = 0 # which out of the 100 trajectories to use
data_path = Path('data')
f_name = 'sim_0134.mat'
S = spio.loadmat(Path(data_path) / f_name)['S']
S_list = [S[0, i] for i in range(S.shape[1])]  # convert to list of structs

# unpack data
n_corr_obs = S_list[sim_idx]['Pc_reshaped'][0, 0] * S_list[sim_idx]['num_trials_mat'][0, 0]
n_total_obs = S_list[sim_idx]['num_trials_mat'][0, 0]
n_frames = S_list[sim_idx]['Pc_reshaped'][0, 0].shape[0]
n_dim = S_list[sim_idx]['x'][0, 0].shape[0]

x, c_est, p, c, d, a, mu_prior_c, mu_post_c = optimize_ELBO(n_dim, n_corr_obs, n_total_obs, verbose=True, n_starts=10)

Running MLE to initialize posterior..........................
Current loss: 413.9496309533715
Loss updated
Iteration 1 | Loss: 413.9496309533715
Current loss: 302.27169866859913
Loss updated
Iteration 2 | Loss: 302.27169866859913
Current loss: 406.47711265832186
Iteration 3 | Loss: 406.47711265832186
Current loss: 273.2588534653187
Loss updated
Iteration 4 | Loss: 273.2588534653187
Current loss: 314.2340337447822
Iteration 5 | Loss: 314.2340337447822
Current loss: 302.59194454550743
Iteration 6 | Loss: 302.59194454550743
Current loss: 276.21732231229544
Iteration 7 | Loss: 276.21732231229544
Current loss: 289.86301036551595
Iteration 8 | Loss: 289.86301036551595
Current loss: 354.1628640219569
Iteration 9 | Loss: 354.1628640219569
Current loss: 309.67359978333116
Iteration 10 | Loss: 309.67359978333116

Start minimizing ELBO..........................


In [ ]:
est_global_curvature = torch.rad2deg(mu_prior_c.detach().numpy())
print(f'Estimated global curvature: {est_global_curvature} degrees')
# print(f'Average estimated local curvature: {torch.mean(c_best).detach().numpy() * (180/np.pi)} degrees')
print(f'Average estimated local curvature: {torch.rad2deg(torch.mean(c).detach().numpy())} degrees')